In [1]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("kingburrito666/shakespeare-plays")
csv_file=os.path.join(path,"Shakespeare_data.csv")
print(csv_file)

100%|██████████| 4.55M/4.55M [00:00<00:00, 31.9MB/s]

Extracting files...


/root/.cache/kagglehub/datasets/kingburrito666/shakespeare-plays/versions/4/Shakespeare_data.csv


In [2]:
import pandas as pd
df = pd.read_csv(csv_file)
df.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [3]:
sentence_df = pd.DataFrame({'sentences': df['PlayerLine']})
sentence_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111396 entries, 0 to 111395
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sentences  111396 non-null  object
dtypes: object(1)
memory usage: 870.4+ KB


In [20]:
import tensorflow as tf
import tensorflow_federated as tff
print(tf.__version__)  # Should print 2.15.0
print(tff.__version__)  # Should print 0.79.0

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [19]:
import tensorflow as tf
import tensorflow_federated as tff

def create_dataset_from_df(sentence_df, column_name='sentences'):

    sentences = sentence_df[column_name].tolist()
    dataset = tf.data.Dataset.from_tensor_slices({'tokens': sentences})
    return dataset

create_dataset_from_df(sentence_df)

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [18]:
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd

def create_dataset_from_df(sentence_df, column_name='sentences'):

    sentences = sentence_df[column_name].tolist()
    dataset = tf.data.Dataset.from_tensor_slices({'tokens': sentences})
    return dataset

def construct_word_level_datasets_from_df(
    sentence_df,
    vocab_size,
    batch_size,
    client_epochs_per_round,
    max_seq_len,
    max_elements_per_user=-1,
    centralized_train=True,
    shuffle_buffer_size=1000,
    num_validation_examples=100,
    num_test_examples=100):

    # Input validation (same as original)
    if batch_size <= 0:
        raise ValueError('batch_size must be a positive integer')
    if client_epochs_per_round <= 0:
        raise ValueError('client_epochs_per_round must be a positive integer')
    if max_seq_len <= 0:
        raise ValueError('max_seq_len must be a positive integer')
    if max_elements_per_user < -1:
        raise ValueError('max_elements_per_user must be an integer at least -1')
    if shuffle_buffer_size is not None and shuffle_buffer_size < 1:
        raise ValueError('shuffle_buffer_size must be an integer greater than 1.')
    if num_validation_examples <= 1:
        raise ValueError('num_validation_examples must be an integer at least 1')
    if num_test_examples is not None and num_test_examples <= 1:
        raise ValueError('num_test_examples must be an integer at least 1')

    # Create dataset from DataFrame
    raw_dataset = create_dataset_from_df(sentence_df)

    # Generate vocabulary from the sentences
    vocab = get_vocab_from_df(sentence_df, vocab_size, column_name='sentences')
    to_ids = build_to_ids_fn(vocab, max_seq_len)

    # Shuffle and take a subset if needed
    def shuffle_and_take(dataset):
        if shuffle_buffer_size:
            dataset = dataset.shuffle(shuffle_buffer_size)
        if max_elements_per_user != -1:
            dataset = dataset.take(max_elements_per_user)
        return dataset

    # Preprocess the dataset
    def preprocess_train(dataset):
        dataset = dataset.map(to_ids)
        if not centralized_train:
            dataset = batch_and_split(
                dataset, max_seq_len, client_epochs_per_round, batch_size)
        return dataset

    # Apply preprocessing
    train = shuffle_and_take(raw_dataset).map(to_ids)
    if centralized_train:
        if shuffle_buffer_size:
            train = train.shuffle(shuffle_buffer_size)
        train = batch_and_split(train, max_seq_len, epochs=client_epochs_per_round, batch_size=batch_size)

    # Split into train, validation, and test
    total_examples = sentence_df.shape[0]
    if num_validation_examples + num_test_examples > total_examples:
        raise ValueError('num_validation_examples + num_test_examples exceeds dataset size')

    validation = train.take(num_validation_examples)
    validation = batch_and_split(validation, max_seq_len)

    test = train.skip(num_validation_examples).take(num_test_examples)
    test = batch_and_split(test, max_seq_len)

    return train, validation, test

def get_vocab_from_df(sentence_df, vocab_size, column_name='sentences'):

    if vocab_size <= 0:
        raise ValueError('vocab_size must be a positive integer')

    sentences = sentence_df[column_name].str.cat(sep=' ')
    words = tf.strings.split(sentences, sep=' ').values
    unique_words, _ = tf.unique_with_counts(words)
    vocab = unique_words[:vocab_size].numpy().tolist()
    return [word.decode('utf-8') for word in vocab]

sentence_df = pd.DataFrame({'sentences': ["hello world", "this is a test", "goodbye world"]})
train, validation, test = construct_word_level_datasets_from_df(
    sentence_df=sentence_df,
    vocab_size=10,
    batch_size=2,
    client_epochs_per_round=1,
    max_seq_len=5,
    num_validation_examples=1,
    num_test_examples=1
)

print(train,validation,test)

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle